In [1]:
%jsroot on

# Population pyramid parametrization
For the sake of simplicity, males and females are assumed to be evenly distributed among all ages, *i.e.* the population pyramid is assumed to be gender-symmetric.  
The pyramid is parametrized with a piecewise linear function, for ages ($x$) between 0 and 100 years:

$$
\mathrm{Pop}(x) = \begin{cases}
m_{1}x, & 0 \leq x < x_{1} \\
m_{2}x+q_{2}, & x_{1} \leq x < x_{2} \\
\cdots, & \cdots \\
m_{n}(x-100), & x_{n-1} \leq x \leq 100
\end{cases}
$$

The pyramid can be defined by specifying couples $\{ \mathrm{age}, \mathrm{weight} \} = \{ a_{i}, w_{i} \}$, in ascending order of age (*i.e.* $a_{i+1} > a_{i}$ for all couples), where $w_{i}$ is the relative importance of the given age with respect to the other given ages.  
For example: $\{ 5, 1 \}$ and $\{ 50, 2 \}$ specifies a pyramid in which people of age 50 are twice more abundant than people of age 5.  
Requiring $\mathrm{Pop}(a_{i}) = w_{i}$ and continuity at all $x = a_{i}$, we can find the values of the parameters $m_{i}$ and $q_{i}$.

## Derivation details
One couple $\{ a_{1}, w_{1} \}$ defines two straight-line segments:

$$
\mathrm{Pop}(x) = \begin{cases}
m_{1}x, & 0 \leq x < a_{1} \\
m_{2}(x-100), & a_{1} \leq x \leq 100
\end{cases}
$$

Requiring $\mathrm{Pop}(a_{1}) = w_{1}$ we get $m_{1}a_{1} = w_{1}$, whence $m_{1} = w_{1}/a_{1}$.  
Continuity in $x = a_{1}$ requires $m_{1}a_{1} = m_{2}(a_{1}-100)$, whence $m_{2} = m_{1}a_{1}/(a_{1}-100) = w_{1}/(a_{1}-100)$.  
The normalization of the distribution is $\int_{0}^{100}\mathrm{Pop}(x)\,dx = 
m_{1}a_{1}^{2}/2 - m_{2}(a_{1}-100)^{2}/2 = 
(1/2)[w_{1}a_{1} - w_{1}(a_{1}-100)^{2}/(a_{1}-100)] =$
$(1/2)w_{1}[a_{1} - (a_{1}-100)] = 50w_{1}$.

Two couples $\{ a_{1}, w_{1} \}$ and $\{ a_{2}, w_{2} \}$ define three straight-line segments:

$$
\mathrm{Pop}(x) = \begin{cases}
m_{1}x, & 0 \leq x < a_{1} \\
m_{2}x+q_{2}, & a_{1} \leq x < a_{2} \\
m_{3}(x-100), & a_{2} \leq x \leq 100
\end{cases}
$$

Requiring $\mathrm{Pop}(a_{1}) = w_{1}$ and $\mathrm{Pop}(a_{2}) = w_{2}$, and continuity in $x = a_{1}$ and $x = a_{2}$, we get $m_{1} = w_{1}/a_{1}$, $m_{2} = (w_{2}-w{1})/(a_{2}-a_{1})$, $m_{3} = w_{2}/(a_{2}-100)$, and $q_{2} = w_{2} - m_{2}a_{2} = (w_{1}a_{2}-w_{2}a_{1})/(a_{2}-a_{1})$.  
The normalization is $\int_{0}^{100}\mathrm{Pop}(x)\,dx = 
m_{1}a_{1}^{2}/2 + (a_{2}-a_{1})[m_{2}(a_{2}+a_{1})+2q_{2}]/2 - m_{3}(a_{2}-100)^{2}/2 =$
$(1/2)\{w_{1}a_{1} + (a_{2}-a_{1})[(w_{2}-w_{1})(a_{2}+a_{1})/(a_{2}-a_{1})+2(w_{1}a_{2} - w_{2}a_{1})/(a_{2}-a_{1})] - w_{2}(a_{2}-100)^{2}/(a_{2}-100)\} = 
(1/2)\{w_{1}a_{1} + w_{2}a_{2} - w_{1}a_{2} + w_{2}a_{1} - w_{1}a_{1} + 2w_{1}a_{2} - 2w_{2}a_{1} - w_{2}a_{2} + 100w_{2}\} =$
$(1/2)\{w_{1}a_{2} + w_{2}(100-a_{1})\}$.

In the general case of $n$ couples $\{ a_{i}, w_{i} \}$, there are $n+1$ segments.  
Let $a_{0} = w_{0} = w_{n} = 0$, $a_{n+1} = 100$.
Then, for $1 \leq i \leq n$, $m_{i} = (w_{i}-w_{i-1})/(a_{i}-a_{i-1})$, $q_{i} = (w_{i-1}a_{i} - w_{i}a_{i-1})/(a_{i}-a_{i-1})$, and the normalization is $(1/2)\sum_{i=1}^{n}w_{i}(a_{i+1}-a_{i-1})$.

In [2]:
vector<pair<double,double>> pop; // population pyramid

// population distribution parameters
vector<double> slopes;
vector<double> intercepts;
double pop_norm;

// create actual distribution
TF1 fpop("fpop", [&](double *x, double *p){ 
    double age = x[0];
    int i = distance(begin(pop),
                     lower_bound(begin(pop), end(pop), age, [](auto a, auto b) { 
                         return a.first < b; 
                     }));
    double y = slopes[i]*age + intercepts[i];
    return y / pop_norm;
}, 0, 100, 0);

In [3]:
pop = { {5,1}, {50,2}, {80,1} };

In [4]:
// compute distribution parameters
{
    slopes.clear();
    intercepts.clear();
    pop_norm = 0;

    // extended population buffer
    auto pop_buf = pop;
    pop_buf.insert(begin(pop_buf), {0, 0});
    pop_buf.push_back({100, 0});
    
    for (int i = 1; i < pop_buf.size(); ++i) {
        auto &pp = pop_buf[i-1];
        auto &p  = pop_buf[i];
        auto &pn = pop_buf[i+1];
        
        slopes.push_back((p.second - pp.second) / (p.first - pp.first));
        intercepts.push_back((pp.second*p.first - p.second*pp.first) / (p.first - pp.first));
        pop_norm += p.second*(pn.first - pp.first);
    }
    pop_norm /= 2;
}

In [5]:
{
    new TCanvas;
    TH1F *h_pop = new TH1F("h_pop", "Population pyramid distribution;Age [years];Number of people", 10, 0, 100);
    h_pop->FillRandom("fpop", 5e4);
    h_pop->SetFillColor(kBlue);
    h_pop->SetBarWidth(0.8);
    h_pop->SetBarOffset(0.1);
    h_pop->Draw("bar");
    pop_norm /= h_pop->GetMaximum()/fpop.GetMaximum();
    fpop.Draw("SAME");
    gPad->SetGrid();
    gPad->Draw();
}